# LLM Inference for Automated ICD-10 Coding (Portuguese)

This notebook runs deterministic (temperature = 0) inference of multiple
large language models (LLMs) for automated ICD-10 coding using
**Portuguese obstetric discharge notes**.

All models are queried via API with identical prompts and decoding settings.
Model outputs are constrained to structured JSON and subsequently normalized
and validated using ICD-10 regex rules.

The resulting predictions are used as input for downstream evaluation,
including performance metrics and non-parametric bootstrap confidence intervals.

This notebook accompanies the paper:
"Large Language Models for Automated ICD-10 Coding of Obstetric Clinical Notes in Portuguese".

Clinical data and model credentials are not distributed with this repository.

##Importando a tabela e selecionando as colunas de interesse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

tabela_auditoria = pd.read_pickle("/content/drive/MyDrive/Doutorado - Ricardo- Após a Qualificação/df_traduzido_gpt_maritalk.pkl")

In [ ]:
tabela_auditoria.columns

In [ ]:
tabela_auditoria = tabela_auditoria[['25. Evolução Alta','21. CID de Alta','25. High Evolution','25. High Evolution (GPT 4o)','25. High Evolution (Maritalk)']]
tabela_auditoria

In [ ]:
cols = [
    "25. Evolução Alta",
    "25. High Evolution",
    "25. High Evolution (GPT 4o)",
    "25. High Evolution (Maritalk)"
]

tabela_auditoria[cols] = (
    tabela_auditoria[cols]
    .fillna("")     # substitui NaN por string vazia
    .astype(str)    # converte tudo para str
)

In [ ]:
tabela_auditoria

##Rodando a GPT 4o - Mini

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
from openai import OpenAI

# 1) Inicializa o cliente OpenAI
client = OpenAI(api_key="APIKEY")

# 2) Modelos Pydantic para validar o JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

# 3) Função que chama a API e retorna lista de strings de CIDs
def extrair_cid(texto: str) -> List[str]:
    if pd.isna(texto) or not texto.strip():
        return []
    resp = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
                "role":"system","content":(
                    "You are a physician specializing in assigning ICD-10 codes "
                    "(International Classification of Diseases) working in a maternal "
                    "and neonatal health hospital. Upon receiving a clinical note, "
                    "identify only the ICD-10 codes most relevant to that note, without "
                    "any explanations or additional information. Use only recognized "
                    "medical guidelines such as WHO, CDC, FDA, or NICE. Read the clinical "
                    "note below and extract:\n"
                    "- 1 primary ICD-10 code\n"
                    "- 1 secondary ICD-10 code\n"
                    "- several tertiary ICD-10 codes\n\n"
                    "Return a VALID JSON in this format (no extra fields):\n"
                    '{\n'
                    '  "cids": [\n'
                    '    { "cid": "O80",   "tipo": "principal"  },\n'
                    '    { "cid": "O99.8", "tipo": "secundário" },\n'
                    '    { "cid": "Z39.0", "tipo": "terciarios" }\n'
                    '  ]\n'
                    '}'
                )
            },
                )
            },
            {"role":"user","content": texto}
        ],
        response_format={"type":"json_object"},
        temperature=0.0,
    )
    json_str = resp.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError:
        return []

# 4) Defina as colunas de input
cols = [
    "25. High Evolution",
    "25. High Evolution (GPT 4o)",
    "25. High Evolution (Maritalk)"
]

# 5) Copia o DataFrame e prepara colunas de input
tabela = tabela_auditoria.copy()
for col in cols:
    tabela[col] = tabela[col].fillna("").astype(str)
    # pré-cria a coluna de saída (dype object)
    tabela[f"API GPT-Mini - {col}"] = None

# 6) Itera linha a linha, extrai, salva (com .at) e imprime para checar
for idx, row in tabela.iterrows():
    print(f"=== Linha {idx} ===")
    for col in cols:
        texto = row[col]
        print(f"\nProcessando coluna {col!r}...")
        cids = extrair_cid(texto)
        # usa .at para atribuir lista diretamente a uma célula
        tabela.at[idx, f"API GPT-Mini - {col}"] = cids
        print("  Texto original:", texto)
        print("  CIDs extraídos:", cids)
    print("\n" + "-" * 40 + "\n")

# 7) Por fim, confira as primeiras linhas
print(tabela.head())

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

# 1) Inicializa o cliente Maritalk
client = OpenAI(
    api_key="APIKEY",
    base_url="https://chat.maritaca.ai/api",
)

# 2) Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

# 3) Função que chama a API Maritalk e retorna lista de strings de CIDs
def extrair_cid(texto: str) -> List[str]:
    if pd.isna(texto) or not texto.strip():
        return []
    resposta = client.chat.completions.create(
        model="sabia-3.1",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a physician specializing in assigning ICD-10 codes "
                    "(International Classification of Diseases) working in a maternal "
                    "and neonatal health hospital. Upon receiving a clinical note, "
                    "identify only the ICD-10 codes most relevant to that note, without "
                    "any explanations or additional information. Use only recognized "
                    "medical guidelines such as WHO, CDC, FDA, or NICE. Read the clinical "
                    "note below and extract:\n"
                    "- 1 primary ICD-10 code\n"
                    "- 1 secondary ICD-10 code\n"
                    "- several tertiary ICD-10 codes\n\n"
                    "Return a VALID JSON in this format (no extra fields):\n"
                    '{\n'
                    '  "cids": [\n'
                    '    { "cid": "O80",   "tipo": "principal"  },\n'
                    '    { "cid": "O99.8", "tipo": "secundário" },\n'
                    '    { "cid": "Z39.0", "tipo": "terciarios" }\n'
                    '  ]\n'
                    '}'
                )
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )
    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Raw response:", json_str)
        return []

# 4) Colunas de input a serem processadas
cols = [
    "25. High Evolution",
    "25. High Evolution (GPT 4o)",
    "25. High Evolution (Maritalk)"
]

# 5) Prepara o DataFrame e cria as colunas de saída
tabela = tabela_auditoria.copy()
for col in cols:
    tabela[col] = tabela[col].fillna("").astype(str)
    tabela[f"API Maritalk - {col}"] = None

# 6) Itera linha a linha, extrai, salva e imprime
for idx, row in tabela.iterrows():
    print(f"=== Linha {idx} ===")
    for col in cols:
        print(f"\nProcessando coluna '{col}'...")
        cids = extrair_cid(row[col])
        tabela.at[idx, f"API Maritalk - {col}"] = cids
        print("  Texto original:", row[col])
        print("  CIDs extraídos:", cids)
    print("\n" + "-" * 40 + "\n")

# 7) Exibe as primeiras linhas para conferência
print(tabela.head())


In [ ]:
tabela

In [ ]:
df = tabela.copy()

In [ ]:
df

#Agora comparando a perfomance de cada um deles continuar daqui

In [ ]:
df = tabela.copy()

# Grava no disco
df.to_pickle('tabela_auditoria.pkl')

# No Colab, dispara o download
from google.colab import files
files.download('tabela_auditoria.pkl')

In [ ]:
colunas_cid = [
    "API Maritalk - 25. High Evolution",
    "API Maritalk - 25. High Evolution (GPT 4o)",
    "API Maritalk - 25. High Evolution (Maritalk)",
]

# Incluindo a coluna "21. CID de Alta"
todas_colunas = colunas_cid + ["21. CID de Alta"]

# Ordena as listas em ordem alfabética
for col in todas_colunas:
    df[col] = df[col].apply(lambda x: sorted(x) if isinstance(x, list) else x)


In [ ]:
import ast

for col in colunas_cid:
    df[col] = df[col].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else x
    )

In [ ]:
import re

padrao_cid = re.compile(r"^[A-Z][0-9]{2}(\.[0-9A-Z]{1,2})?$")

def filtrar_cids(lista):
    if not isinstance(lista, list):
        return []
    return [item for item in lista if isinstance(item, str) and padrao_cid.match(item)]

for col in colunas_cid:
    df[col] = df[col].apply(filtrar_cids)

In [ ]:
df

In [ ]:
!pipinstallcollections
import pandas as pd
from collections import defaultdic


code2cluster = {cat: cid for cid, cats in clusters.items() for cat in cats}

# -------------------------------------------------------------------
# 3) Funções de ancestrais
# -------------------------------------------------------------------
def get_ancestors_basic(cid_code):
    level2 = cid_code
    level1 = cid_code.split('.')[0]
    level0 = level1[0]
    return [level2, level1, level0]

def get_ancestors_cluster(cid_code):
    level2 = cid_code
    level1 = cid_code.split('.')[0]
    cluster_id = code2cluster.get(level1)
    level0 = f'cluster_{cluster_id}' if cluster_id is not None else 'cluster_unk'
    return [level2, level1, level0]

# -------------------------------------------------------------------
# 4) Função para calcular micro CoPHE em um dicionário gold/pred
# -------------------------------------------------------------------
def compute_cophe(gold, pred, mode='basic'):
    total_TP = total_FP = total_FN = 0
    for d in gold:
        x_counts, y_counts = defaultdict(int), defaultdict(int)
        codes_pred = pred[d]
        codes_gold = gold[d]
        for code in codes_pred:
            anc_list = (get_ancestors_basic if mode=='basic' else get_ancestors_cluster)(code)
            for anc in anc_list:
                x_counts[anc] += 1
        for code in codes_gold:
            anc_list = (get_ancestors_basic if mode=='basic' else get_ancestors_cluster)(code)
            for anc in anc_list:
                y_counts[anc] += 1
        for c in set(x_counts) | set(y_counts):
            x, y = x_counts[c], y_counts[c]
            TP, FP, FN = min(x,y), max(x-y,0), max(y-x,0)
            total_TP += TP; total_FP += FP; total_FN += FN
    P = total_TP/(total_TP+total_FP) if total_TP+total_FP else 0
    R = total_TP/(total_TP+total_FN) if total_TP+total_FN else 0
    F1 = 2*P*R/(P+R) if (P+R) else 0
    return P, R, F1

# -------------------------------------------------------------------
# 5) Função para calcular macro CoPHE (média por documento)
# -------------------------------------------------------------------
def compute_cophe_macro(gold, pred, mode='basic'):
    precisions, recalls, f1s = [], [], []
    for d in gold:
        # micro-por-documento:
        p, r, f1 = compute_cophe({d: gold[d]}, {d: pred[d]}, mode=mode)
        precisions.append(p)
        recalls.append(r)
        f1s.append(f1)
    return (sum(precisions)/len(precisions),
            sum(recalls)/len(recalls),
            sum(f1s)/len(f1s))

# -------------------------------------------------------------------
# 6) Calcular micro e macro para cada API
# -------------------------------------------------------------------
pred_cols = [c for c in df.columns if c.startswith('API')]
results = []

gold_dict = {idx: codes for idx, codes in df['21. CID de Alta'].items()}

for api in pred_cols:
    pred_dict = {idx: codes for idx, codes in df[api].items()}
    # micro
    bp_mi, br_mi, bf_mi = compute_cophe(gold_dict, pred_dict, mode='basic')
    ap_mi, ar_mi, af_mi = compute_cophe(gold_dict, pred_dict, mode='cluster')
    # macro
    bp_ma, br_ma, bf_ma = compute_cophe_macro(gold_dict, pred_dict, mode='basic')
    ap_ma, ar_ma, af_ma = compute_cophe_macro(gold_dict, pred_dict, mode='cluster')
    results.append({
        'Model': api,
        'Basic P (micro)': bp_mi, 'Basic R (micro)': br_mi, 'Basic F1 (micro)': bf_mi,
        'Adapted P (micro)': ap_mi,'Adapted R (micro)': ar_mi,'Adapted F1 (micro)': af_mi,
        'Basic P (macro)': bp_ma, 'Basic R (macro)': br_ma, 'Basic F1 (macro)': bf_ma,
        'Adapted P (macro)': ap_ma,'Adapted R (macro)': ar_ma,'Adapted F1 (macro)': af_ma,
    })

metrics_df = pd.DataFrame(results).set_index('Model')
print(metrics_df.to_markdown())

In [ ]:
def comparar_listas(coluna_predita, coluna_referencia="21. CID de Alta"):
    intersecoes = []
    faltando = []
    extras = []

    for idx, linha in df.iterrows():
        ref = set(linha[coluna_referencia]) if isinstance(linha[coluna_referencia], list) else set()
        pred = set(linha[coluna_predita]) if isinstance(linha[coluna_predita], list) else set()

        intersecoes.append(sorted(ref & pred))      #  IGUAIS
        faltando.append(sorted(ref - pred))         #  FALTANDO
        extras.append(sorted(pred - ref))           #  EXTRAS

    df[f'{coluna_predita} - IGUAIS'] = intersecoes
    df[f'{coluna_predita} - FALTANDO'] = faltando
    df[f'{coluna_predita} - EXTRAS'] = extras

In [ ]:
for col in colunas_cid:
    comparar_listas(col)


In [ ]:
def ensure_list(x):
    if isinstance(x, list): return x
    if pd.isna(x):       return []
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return [i.strip() for i in x.split(',') if i.strip()]
    return []

# 3) Converta a verdade (CID_Alta) e cada API em listas
df['CID_Alta_list'] = df['CID_Alta'].apply(ensure_list)
for col in api_cols:
    df[f'{col}_list'] = df[col].apply(ensure_list)

# 4) Monte o summary apenas para essas seis APIs
summary = []
for col in api_cols:
    tp   = df.apply(lambda r: len(set(r['CID_Alta_list']) & set(r[f'{col}_list'])), axis=1)
    pred = df[f'{col}_list'].str.len()
    gold = df['CID_Alta_list'].str.len()
    fp   = pred - tp
    fn   = gold - tp

    # métrica por instância
    prec = tp / (tp+fp).replace({0:np.nan})
    rec  = tp / (tp+fn).replace({0:np.nan})
    denom= (prec+rec).replace({0:np.nan})
    f1   = 2*prec*rec/denom
    jacc = tp / (tp+fp+fn).replace({0:np.nan})

    # agregados micro
    TP,FP,FN = tp.sum(), fp.sum(), fn.sum()
    prec_micro = TP/(TP+FP) if TP+FP else 0
    rec_micro  = TP/(TP+FN) if TP+FN else 0
    f1_micro   = 2*prec_micro*rec_micro/(prec_micro+rec_micro) if (prec_micro+rec_micro) else 0
    jacc_micro = TP/(TP+FP+FN) if (TP+FP+FN) else 0

    # agregados macro
    prec_macro, rec_macro = prec.mean(), rec.mean()
    f1_macro, jacc_macro = f1.mean(),  jacc.mean()

    # outras
    hamming   = (fp+fn).sum()/(pred+gold).sum()
    card_diff = (pred-gold).abs().mean()
    exact     = ((tp==gold)&(pred==gold)).mean()

    summary.append({
        'API':             col,
        'Prec (macro)':    prec_macro,
        'Rec (macro)':     rec_macro,
        'F1 (macro)':      f1_macro,
        'Jaccard (macro)': jacc_macro,
        'Prec (micro)':    prec_micro,
        'Rec (micro)':     rec_micro,
        'F1 (micro)':      f1_micro,
        'Jaccard (micro)': jacc_micro,
        'Hamming loss':    hamming,
        'Card. diff.':     card_diff,
        'Exact match %':   exact
    })

summary_df = pd.DataFrame(summary)

# 5) Exiba só as seis linhas
print(summary_df)

##GPT Mini

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

# Inicialize o cliente OpenAI
client = OpenAI(api_key="APIKEY")

# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:
    """
    Chama o modelo fine-tuned e retorna a lista de CIDs extraídos.
    """
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []

tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplica a função de extração de CID em cada nota clínica
tabela_auditoria["API GPT-Mini"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)

# Exibe as primeiras linhas para conferência
print(tabela_auditoria.head())




In [ ]:
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("").astype(str)

In [ ]:
tabela_auditoria

##Rodando GPT - 4o

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

# Inicialize o cliente OpenAI
client = OpenAI(api_key="APIKEY")

# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:
    """
    Chama o modelo fine-tuned e retorna a lista de CIDs extraídos.
    """
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []


tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = (
    tabela_auditoria["25. Evolução Alta"].fillna("")
)

tabela_auditoria["API GPT‑4o"] = (
    tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)
)

print(tabela_auditoria.head())



In [ ]:
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("").astype(str)

In [ ]:
tabela_auditoria

##Rodando Sabia 3

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import openai
from openai import OpenAI

client = openai.OpenAI(
    api_key="APIKEY",
    base_url="https://chat.maritaca.ai/api",
)
# Modelos Pydantic para validação do JSON de saída
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]

def extrair_cid(texto: str) -> List[str]:

    if pd.isna(texto) or str(texto).strip() == "":
        return []

    resposta = client.chat.completions.create(
        model="sabia-3.1",
        messages=[
            {
                "role": "system",
                "content": """Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {"role": "user", "content": texto},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )

    json_str = resposta.choices[0].message.content
    try:
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]
    except ValidationError as e:
        print("ValidationError:", e)
        print("Resposta bruta:", json_str)
        return []


tabela_auditoria = tabela_auditoria.copy()
tabela_auditoria["25. Evolução Alta"] = (
    tabela_auditoria["25. Evolução Alta"].fillna("")
)

tabela_auditoria["API Maritalk"] = (
    tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)
)

print(tabela_auditoria.head())

In [ ]:
tabela_auditoria

##Rodando DeepSeek

In [ ]:
import pandas as pd
import requests
from typing import List
import numpy as np

# Configuração da API
API_KEY = "APIKEY"
API_URL = "https://api.deepseek.com/v1/chat/completions"

def extrair_cid(texto) -> List[str]:
    # Garantir que o valor seja sempre uma string
    texto = str(texto) if not isinstance(texto, str) else texto

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "deepseek-chat",
        "messages": [
            {
                "role": "system",
                "content":
                    """"Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças)
que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota,
sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{
  "cids": [
    { "cid": "O80",    "tipo": "principal"  },
    { "cid": "O99.8",  "tipo": "secundário" },
    { "cid": "Z39.0",  "tipo": "terciarios" },
    { "cid": "F53.0",  "tipo": "terciarios" }
  ]
}

Nada além desse JSON — sem comentários ou campos extras.
"""
            },
            {
                "role": "user",
                "content": texto
            }
        ],
        "temperature": 0.1
    }

    try:
        response = requests.post(API_URL, json=payload, headers=headers)
        response.raise_for_status()

        # Processamento da resposta
        resposta = response.json()
        conteudo = resposta["choices"][0]["message"]["content"]

        # Extrai códigos (ex.: "A15.0, J18.9" → ["A15.0", "J18.9"])
        return [cid.strip() for cid in conteudo.split(",") if cid.strip()]

    except Exception as e:
        # Convertendo novamente para string, caso necessário, para o slice
        texto_str = str(texto)
        print(f"Erro ao processar '{texto_str[:30]}...': {e}")
        return []


# Substituir valores NaN por string vazia para evitar problemas
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplicar a função no DataFrame
tabela_auditoria["API Deep-Seek - Sem Prompt"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid)

print(tabela_auditoria)


In [ ]:
tabela_auditoria

##API GEMINI

In [ ]:
import pandas as pd
from typing import List, Literal
from pydantic import BaseModel, ValidationError
import google.generativeai as genai
import json
import os


try:
    # Substitua pela sua chave de API do Google AI Studio
    GOOGLE_API_KEY = "APIKEY"
    genai.configure(api_key=GOOGLE_API_KEY)
except Exception as e:
    print(f"Erro ao configurar a API do Gemini: {e}")
    # O script não poderá continuar sem a configuração da API.
    exit()


# --- Modelos Pydantic para Validação (sem alterações) ---
class CIDItem(BaseModel):
    cid: str
    tipo: Literal["principal", "secundário", "terciarios"]

class UltrasoundCIDExtraction(BaseModel):
    cids: List[CIDItem]


# --- Função de Extração Adaptada para o Gemini 1.5 Flash ---
def extrair_cid_gemini(texto: str) -> List[str]:
    if pd.isna(texto) or str(texto).strip() == "":
        return []

    # Configuração do modelo Gemini
    # Usamos o response_mime_type para forçar a saída em JSON.
    generation_config = {
        "temperature": 0.0,
        "response_mime_type": "application/json",
    }

    # O modelo generativo que será usado
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config
    )

    # O prompt é montado em uma única string, combinando as instruções e o texto do usuário.
    prompt_completo = f"""
Você é um médico especializado na determinação de códigos CID-10 (Classificação Internacional de Doenças) que atua em um hospital de saúde materna e neonatal.
Ao receber uma nota clínica, identifique exclusivamente os códigos CID-10 mais associados àquela nota, sem qualquer informação ou explicação adicional.
Utilize apenas diretrizes médicas reconhecidas (OMS, CDC, FDA, NICE).

Leia atentamente a nota clínica abaixo e extraia:
- 1 CID “principal” (motivo principal de internação/alta);
- 1 CID “secundário” (complicações ou comorbidades relevantes);
- Vários CIDs “terciarios” (demais condições associadas).

Sua resposta deve ser UM JSON VÁLIDO no formato de exemplo:

{{
  "cids": [
    {{ "cid": "O80",    "tipo": "principal"  }},
    {{ "cid": "O99.8",  "tipo": "secundário" }},
    {{ "cid": "Z39.0",  "tipo": "terciarios" }},
    {{ "cid": "F53.0",  "tipo": "terciarios" }}
  ]
}}

Nada além desse JSON — sem comentários, explicações ou campos extras.

---
NOTA CLÍNICA PARA ANÁLISE:
{texto}
---
"""

    try:
        # Chamada para a API do Gemini
        resposta = model.generate_content(prompt_completo)

        # O resultado já vem como um objeto de texto, pronto para ser validado.
        json_str = resposta.text

        # Validação com Pydantic
        obj = UltrasoundCIDExtraction.model_validate_json(json_str)
        return [item.cid for item in obj.cids]

    except ValidationError as e:
        print(f"Erro de validação do Pydantic: {e}")
        print(f"Resposta bruta recebida: {json_str}")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao chamar a API do Gemini: {e}")
        return []



# Substituir valores NaN por string vazia para evitar problemas
tabela_auditoria["25. Evolução Alta"] = tabela_auditoria["25. Evolução Alta"].fillna("")

# Aplicar a função no DataFrame
tabela_auditoria["API Gemini"] = tabela_auditoria["25. Evolução Alta"].apply(extrair_cid_gemini)

print(tabela_auditoria)

In [ ]:
tabela_auditoria.loc[1].to_frame().T
